### computer_vision_project.ipynb

In [34]:
import tensorflow_datasets as tfds

In [33]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## TensorFlow Datasets
### 사전에 만들어 놓은 머신러닝 데이터셋을 모아둔 것

* 텐서로 이미 데이터를 준비
* 잘 준비된 데이터 정리된 데이터로 실습을 할 수 있다
* 여러가지 데이터 불러오는 방법을 실습할 수 있다.
* 텐서플로 버전이 바뀌면서 새롭게 생기거나 변경된 경우에 빠르게 이 데이터셋으로 테스트를 할 수 있다. 

In [35]:
dataset_list = tfds.list_builders()
print("food101" in dataset_list)

True


In [36]:
(train_data, test_data), dataset_info = tfds.load(
    name="food101", # 우리가 사용할 데이터셋 지정
    split=["train", "validation"],  # 데이터셋 나누는 부분
    shuffle_files=True,  # 파일들을 섞어 줄것인지 default = False
    as_supervised=True, # tuple 형태 (data, label) 로 데이터를 다운로드 할 것인지 여부 / False면 dictionary 형태
    with_info=True, # 데이터셋의 메타 정보도 다운로드 (라벨, 샘플의 갯수 등등)
    download=False)

### Windows의 경우 C:\Users\[자신의 아이디]\tensorflow_datasets 에 다운로드 한 후 압축 해제

In [37]:
dataset_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [38]:
class_names = dataset_info.features["label"].names
class_names[:20]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla']

In [39]:
train_one_sample = train_data.take(1) # (image_tensor, label) 왜? as_supervised=True 로 설정을 했기 때문에

In [40]:
train_one_sample

<TakeDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [41]:
type(train_one_sample)

tensorflow.python.data.ops.dataset_ops.TakeDataset

In [42]:
for image, label in train_one_sample:
    print(f"Image shape : {image.shape}")
    print(f"Image dtype : {image.dtype}")
    print(f"Target class : {label}")
    print(f"Class name : {class_names[label.numpy()]}")

Image shape : (512, 512, 3)
Image dtype : <dtype: 'uint8'>
Target class : 31
Class name : donuts


In [43]:
image

<tf.Tensor: shape=(512, 512, 3), dtype=uint8, numpy=
array([[[131,  86,  55],
        [134,  89,  58],
        [137,  92,  61],
        ...,
        [131,  44,   0],
        [135,  49,   2],
        [138,  52,   5]],

       [[134,  89,  58],
        [135,  90,  59],
        [135,  90,  59],
        ...,
        [133,  46,   1],
        [134,  48,   1],
        [134,  48,   1]],

       [[136,  91,  60],
        [137,  92,  61],
        [139,  94,  63],
        ...,
        [137,  53,   7],
        [135,  51,   5],
        [131,  47,   1]],

       ...,

       [[130,  23,   3],
        [134,  31,  14],
        [132,  36,  24],
        ...,
        [243, 248, 242],
        [243, 248, 242],
        [243, 248, 242]],

       [[128,  22,   0],
        [129,  26,   7],
        [124,  28,  14],
        ...,
        [243, 248, 241],
        [242, 247, 240],
        [242, 247, 240]],

       [[132,  26,   2],
        [133,  31,   9],
        [126,  29,  13],
        ...,
        [242, 247, 24

In [44]:
type(image)

tensorflow.python.framework.ops.EagerTensor

In [45]:
image.ndim

3

In [46]:
import tensorflow as tf

In [47]:
tf.reduce_min(image), tf.reduce_max(image)

(<tf.Tensor: shape=(), dtype=uint8, numpy=0>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=255>)

In [48]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)

(-0.5, 511.5, 511.5, -0.5)

In [49]:
def preprocess_img(image, label, img_shape = 224):
    image = tf.image.resize(image, [img_shape, img_shape])
    return tf.cast(image, tf.float32), label

In [50]:
preprocessed_img = preprocess_img(image, label)[0]

In [51]:
print(f"{image[:2]} {image.shape} {image.dtype}")
print(f"{preprocessed_img[:2]} {preprocessed_img.shape} {preprocessed_img.dtype}")

[[[131  86  55]
  [134  89  58]
  [137  92  61]
  ...
  [131  44   0]
  [135  49   2]
  [138  52   5]]

 [[134  89  58]
  [135  90  59]
  [135  90  59]
  ...
  [133  46   1]
  [134  48   1]
  [134  48   1]]] (512, 512, 3) <dtype: 'uint8'>
[[[1.34030609e+02 8.90306168e+01 5.80306129e+01]
  [1.36045914e+02 9.10459213e+01 6.00459175e+01]
  [1.37846939e+02 9.28469391e+01 6.38469391e+01]
  ...
  [1.30265366e+02 4.79183846e+01 1.62753057e+00]
  [1.30760223e+02 4.37602272e+01 4.59267795e-02]
  [1.34739838e+02 4.87398338e+01 1.73983324e+00]]

 [[1.41551025e+02 9.65510254e+01 6.55510254e+01]
  [1.49081650e+02 1.04081635e+02 7.30816345e+01]
  [1.51632660e+02 1.06632652e+02 7.56326523e+01]
  ...
  [1.30357101e+02 4.85713654e+01 1.99989128e+00]
  [1.34147995e+02 5.01479988e+01 4.14799976e+00]
  [1.33571289e+02 4.95712891e+01 3.57128906e+00]]] (224, 224, 3) <dtype: 'float32'>


In [52]:
print(f"{image.shape}")
print(f"{preprocessed_img.shape}")

(512, 512, 3)
(224, 224, 3)


In [53]:
print(f"{image.dtype}")
print(f"{preprocessed_img.dtype}")

<dtype: 'uint8'>
<dtype: 'float32'>


In [54]:
plt.imshow(preprocessed_img/255.)
plt.title(class_names[label])
plt.axis(False);

### 현재 우리가 가지고 있는 이미지 데이터는 총 101,000개 입니다. (학습용 및 테스트용 모두 합친 것)
### batch를 잘 고려해야하는 상황! => 메모리를 효율적으로 사용할 수 있기 때문에!

### 101,000개 이미지와 라벨 정보를 32개 이미지와 라벨 정보로 구분해서 처리

### tf.data API의 메서드들을 사용!

[tf data API](https://www.tensorflow.org/guide/data_performance)


* map()
* shuffler()
* batch()
* prefetch()
* cache()

## 배치로 처리할 때 고려해야 할 사항
1. 다른 shape의 텐서들을 배치로 처리할 수 없습니다. 이미지 resize와 같은 것을 처리!

2. shuffle()는 섞을 이미지의 수를 유지를 합니다. 이 때 shuffle의 대상은 학습 데이터의 모두 이미지가 대상

   학습 데이터의 크기가 크다면, 메모리에 효율적 사용이 어렵다. 1,000이나 10,000로 사용하는 것도 나쁘지는 않다.


3. num_parallel_calls 파라미터를 사용할 수 있는 메서드에서는 성능을 개선시킬 수 있습니다. num_parallel_calls = tf.data.AUTORUNE 설정!


4. 데이터셋이 메모리게 잘 맞으면 cache()를 사용할 수 없습니다.

![이미지](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/07-prefetching-from-hands-on-ml.png)

## 일반적인 작업 순서
origianl dataset -> map() -> shuffle() -> batch() -> prefetch() -> PrefetchDataset

In [55]:
train_data = train_data.map(
    map_func = preprocess_img,
    num_parallel_calls = tf.data.AUTOTUNE
)

train_data = train_data.shuffle(buffer_size = 1000).batch(batch_size=32).prefetch(tf.data.AUTOTUNE)

test_data = test_data.map(
    map_func = preprocess_img,
    num_parallel_calls = tf.data.AUTOTUNE
)

test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [56]:
train_data, test_data

(<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>,
 <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>)

In [57]:
checkpoint_path = "model_checkpoint/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor = "val_accuracy",
    save_best_only = True,
    save_weight_only = True,
    verbose = 1
)

In [58]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy = "mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


## mixed precision 학습
### 일반적으로 tensorflow의 텐서는 기본적으로 float32 데이터형!
### computer science에서는 float32를 single-precision floating-point 형식, 32비트를 컴퓨터 메모리에서 사용한다는 것을 의미
### GPU에는 제한된 메모리를 가지고 있습니다. 그래서 GPU 동시에 float32 형식의 텐서들만을 처리할 수 있습니다.
### Mixed Precision 학습은 제한된 메모리를 가진 GPU에서 메모리를 더 잘 활용하기 위해 float16, float32을 혼합해서 사용하는 방식입니다.
### float16은 16비트로 숫자를 표현하는 것으로 float32의 메모리 사용의 반으로 숫자를 표현!
### GPU를 활용해서 혼합 정밀도 학습을 사용하면 최대 3배까지 성능을 향상시킬 수 있다.
### GPU 점수가 7.0 이상이어야 혼합 정밀도 학습이 작동합니다.

[Mixed Precision 학습](https://www.tensorflow.org/guide/mixed_precision#supported_hardware)


In [59]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy = "float32") # mixed_float16

In [60]:
mixed_precision.global_policy()

<Policy "float32">

### Callback도 준비해 놓았고,
### Mixed Precision도 설정해 놓았고,
### 이제 모델을 만들면 됩니다.

## 현재 우리 데이터셋은 아주 큽니다. food101은 용량이 5GB정도입니다.
### 전이학습 (EfficientNetB0)으로 fine-tuning을 하려합니다.

### 일반적인 전이학습의 단계
1. feature extraction 모델 구축 (사전 훈련한 모델의 상위 몇 개 레이어 교체)
2. 하위 레이어가 고정된 epoch 동안 학습
3. 필요한 경우 unfrozen으로 레이어의 상태를 바꾸어서 fine-tuning을 실행

In [61]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

In [62]:
inputs = layers.Input(shape=input_shape, name="input_layer")
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name = "pooling_layer")(x)
x = layers.Dense(len(class_names))(x)

outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(
    loss = "sparse_categorical_crossentropy",  # 라벨의 상태가 one-hot encoding 상태가 sparse_categorical_corssentropy
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [63]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layer (GlobalAverage (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               129381    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [64]:
for layer in model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_layer True float32 <Policy "float32">
efficientnetb0 False float32 <Policy "float32">
pooling_layer True float32 <Policy "float32">
dense True float32 <Policy "float32">
softmax_float32 True float32 <Policy "float32">


In [65]:
for layer in model.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 False float32 <Policy "float32">
rescaling False float32 <Policy "float32">
normalization False float32 <Policy "float32">
stem_conv_pad False float32 <Policy "float32">
stem_conv False float32 <Policy "float32">
stem_bn False float32 <Policy "float32">
stem_activation False float32 <Policy "float32">
block1a_dwconv False float32 <Policy "float32">
block1a_bn False float32 <Policy "float32">
block1a_activation False float32 <Policy "float32">
block1a_se_squeeze False float32 <Policy "float32">
block1a_se_reshape False float32 <Policy "float32">
block1a_se_reduce False float32 <Policy "float32">
block1a_se_expand False float32 <Policy "float32">
block1a_se_excite False float32 <Policy "float32">
block1a_project_conv False float32 <Policy "float32">
block1a_project_bn False float32 <Policy "float32">
block2a_expand_conv False float32 <Policy "float32">
block2a_expand_bn False float32 <Policy "float32">
block2a_expand_activation False float32 <Policy "float32">


In [34]:

history_101_food_classes_feature_extract = model.fit(
    train_data,
    epochs = 3,
    steps_per_epoch= len(train_data),
    validation_data = test_data,
    validation_steps = int(0.15 * len(test_data)),
    callbacks = [
        create_tensorboard_callback("traning_logs", "efficientnetb0_101_classes_all_data_feature_extract"),
        model_checkpoint
    ]
)


TensorBoard 로그 파일을 저장한 디렉토리 : traning_logs/efficientnetb0_101_classes_all_data_feature_extract/20210706-121706
Epoch 1/3
2368/2368 [==============================] - 3738s 2s/step - loss: 1.8203 - accuracy: 0.5570 - val_loss: 1.2229 - val_accuracy: 0.6790

Epoch 00001: val_accuracy improved from -inf to 0.67903, saving model to model_checkpoint/cp.ckpt
INFO:tensorflow:Assets written to: model_checkpoint/cp.ckpt/assets
INFO:tensorflow:Assets written to: model_checkpoint/cp.ckpt/assets
Epoch 2/3
2368/2368 [==============================] - 4542s 2s/step - loss: 1.2919 - accuracy: 0.6670 - val_loss: 1.1348 - val_accuracy: 0.6997

Epoch 00002: val_accuracy improved from 0.67903 to 0.69968, saving model to model_checkpoint/cp.ckpt
INFO:tensorflow:Assets written to: model_checkpoint/cp.ckpt/assets
INFO:tensorflow:Assets written to: model_checkpoint/cp.ckpt/assets
Epoch 3/3
2368/2368 [==============================] - 5626s 2s/step - loss: 1.1405 - accuracy: 0.7030 - val_loss: 1.0866 - val_ac

In [35]:
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

790/790 [==============================] - 1290s 2s/step - loss: 1.0811 - accuracy: 0.7072


[1.081135630607605, 0.7071683406829834]

In [36]:
## 잘 만든 모델이 있다!

In [37]:
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layer (GlobalAverage (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               129381    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [38]:
checkpoint_path

'model_checkpoint/cp.ckpt'

In [39]:
cloned_model.load_weights(checkpoint_path)

In [42]:
cloned_model.compile(
    loss = "sparse_categorical_crossentropy",  # 라벨의 상태가 one-hot encoding 상태가 sparse_categorical_corssentropy
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [43]:
results_feature_extract_model_with_loaded_weights = cloned_model.evaluate(test_data)

790/790 [==============================] - 1171s 1s/step - loss: 1.0811 - accuracy: 0.7072


In [44]:
import numpy as np
assert np.isclose(
    results_feature_extract_model,
    results_feature_extract_model_with_loaded_weights    
).all()

In [45]:
for layer in cloned_model.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling False float32 <Policy "float32">
normalization False float32 <Policy "float32">
stem_conv_pad False float32 <Policy "float32">
stem_conv False float32 <Policy "float32">
stem_bn False float32 <Policy "float32">
stem_activation False float32 <Policy "float32">
block1a_dwconv False float32 <Policy "float32">
block1a_bn False float32 <Policy "float32">
block1a_activation False float32 <Policy "float32">
block1a_se_squeeze False float32 <Policy "float32">
block1a_se_reshape False float32 <Policy "float32">
block1a_se_reduce False float32 <Policy "float32">
block1a_se_expand False float32 <Policy "float32">
block1a_se_excite False float32 <Policy "float32">
block1a_project_conv False float32 <Policy "float32">
block1a_project_bn False float32 <Policy "float32">
block2a_expand_conv False float32 <Policy "float32">
block2a_expand_bn False float32 <Policy "float32">
block2a_expand_activation False float32 <Policy "float32">


In [46]:
save_dir = "efficientnetb0_feature_extract_model"
model.save(save_dir)

INFO:tensorflow:Assets written to: efficientnetb0_feature_extract_model/assets
INFO:tensorflow:Assets written to: efficientnetb0_feature_extract_model/assets


In [49]:
# 저장한 모델을 가져와기
# from keras.models import load_model

new_model = tf.keras.models.load_model(save_dir)

ion (__inference_block3a_expand_activation_layer_call_and_return_conditional_losses_203920) with ops with custom gradients. Will likely fail if a gradient is requested.


In [50]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layer (GlobalAverage (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               129381    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
# dtype_policy를 유지하고 있는지 확인

In [ ]:
# 불러온 모델의 성능을 확인해보세요!

In [ ]:
# 불러온 모델의 성능이 이전 모델의 성능과 같은 비교